In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from collections import defaultdict
from copy import deepcopy
import json
import os
import warnings
import glob
import re
from io import StringIO, BytesIO
from functools import partial
from typing import Tuple

import h5py
import k3d
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange

In [10]:
import sys
sys.path.append('../')

In [11]:
from gcv_v20211_hw1.utils.plotting import display_depth_sharpness, display_sharpness
from gcv_v20211_hw1.utils.hdf5.dataset import Hdf5File, PreloadTypes
from gcv_v20211_hw1.utils.sharpf_io import WholeDepthMapIO, PointPatchPredictionsIO
from gcv_v20211_hw1.utils.camera_utils.camera_pose import camera_to_display, CameraPose

In [12]:
sys.path

['/home/denis/GeometricCV/skoltech-gcv-course-2021/gcv-homework/gcv_v2021.1_hw1/notebooks',
 '/home/denis/miniconda3/envs/geometriccv/lib/python37.zip',
 '/home/denis/miniconda3/envs/geometriccv/lib/python3.7',
 '/home/denis/miniconda3/envs/geometriccv/lib/python3.7/lib-dynload',
 '',
 '/home/denis/miniconda3/envs/geometriccv/lib/python3.7/site-packages',
 '/home/denis/miniconda3/envs/geometriccv/lib/python3.7/site-packages/IPython/extensions',
 '/home/denis/.ipython',
 '..',
 '../']

In [8]:
ground_truth_dataset = Hdf5File(
    '/Users/artonson/tmp/gcv_hw1_data/abc_0050_00500149_54930d6f7740b03347d89a56_000.hdf5',
    io=WholeDepthMapIO,
    preload=PreloadTypes.LAZY,
    laabels='*')


predictions_dataset = Hdf5File(
    '/Users/artonson/tmp/gcv_hw1_data/abc_0050_00500149_54930d6f7740b03347d89a56_000__predictions.hdf5',
    io=WholeDepthMapIO,
    preload=PreloadTypes.LAZY,
    labels='*')

OSError: Unable to open file (unable to open file: name = '/Users/artonson/tmp/gcv_hw1_data/abc_0050_00500149_54930d6f7740b03347d89a56_000.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
def distances_nobg(distances, image):
    distances_ = np.zeros_like(distances)
    distances_[np.nonzero(image)] = distances[np.nonzero(image)]
    return distances_

In [ ]:
display_depth_sharpness(
    depth_images=[
        camera_to_display(ground_truth_dataset[0]['image']),
        camera_to_display(ground_truth_dataset[1]['image']),
        camera_to_display(ground_truth_dataset[2]['image']),
    ],
    sharpness_images=[
        camera_to_display(
            distances_nobg(predictions_dataset[0]['distances'], 
                           ground_truth_dataset[0]['image'])),
        camera_to_display(
            distances_nobg(predictions_dataset[1]['distances'], 
                           ground_truth_dataset[1]['image'])),
        camera_to_display(
            distances_nobg(predictions_dataset[2]['distances'], 
                           ground_truth_dataset[2]['image']))
    ],
    axes_size=(8, 8),
    ncols=3)

In [ ]:
fused_gt_dataset = Hdf5File(
    '/Users/artonson/tmp/gcv_hw1_data/abc_0050_00500149_54930d6f7740b03347d89a56_000__ground_truth.hdf5',
    io=PointPatchPredictionsIO,
    preload=PreloadTypes.LAZY,
    labels='*')

fused_pred_dataset = Hdf5File(
    '/Users/artonson/tmp/gcv_hw1_data/abc_0050_00500149_54930d6f7740b03347d89a56_000__interpolated.hdf5',
    io=PointPatchPredictionsIO,
    preload=PreloadTypes.LAZY,
    labels='*')

In [ ]:
display_sharpness(
    samples=fused_gt_dataset[0]['points'],
    samples_distances=fused_pred_dataset[0]['distances']
)